In [1]:

import math, random
from dataclasses import dataclass
from typing import Iterable, Optional, Tuple, List

import pandas as pd
import matplotlib.pyplot as plt

In [2]:
DT = 0.1  # keep in sync with TwinParams.dt

#### VFD Computations and using @dataclass

In [10]:
# Parameters & twin

@dataclass
class VFDParams:
    """Holds the electrical and speed ratings necessary for V/Hz control."""
    
    # --- Necessary VFD/Motor Ratings (from your 460V/60Hz problem) ---
    rated_voltage_V: float = 460.0  # Motor's base/rated voltage (V)
    rated_frequency_Hz: float = 60.0 # Motor's base/rated frequency (Hz)
    rated_rpm: float = 2000.0       # Motor's rated/base speed (RPM)
    
    # --- Other Twin/Control Parameters (from your previous class) ---
    min_rpm: float = 800.0
    max_rpm: float = 1800.0
    Kp: float = 8.0
    Ki: float = 1.0
    # dt: float = DT # Assuming DT is a defined constant elsewhere
    vfd_ramp_rate_rpm_s: float = 2000.0 
    rated_power_kw: float = 15.0

In [11]:
def calculate_vfd_voltage_dataclass(target_rpm: float, params: VFDParams) -> tuple[float, float]:

    # Access the rated parameters from the dataclass object
    rated_voltage = params.rated_voltage_V
    rated_frequency = params.rated_frequency_Hz
    rated_rpm = params.rated_rpm
    
    # 1. Calculate the Rated V/Hz Ratio (Base V/Hz)
    if rated_frequency == 0:
        # Avoid division by zero
        return 0.0, 0.0
        
    v_per_hz_ratio = rated_voltage / rated_frequency
    
    # 2. Determine the Target Frequency (f_target)
    # Target frequency is scaled linearly with the ratio of target RPM to rated RPM.
    # f_target = f_rated * (rpm_target / rpm_rated)
    target_frequency = rated_frequency * (target_rpm / rated_rpm)
    
    # 3. Calculate the Target Voltage (V_set)
    # V_set = V/Hz_ratio * f_target
    v_set = v_per_hz_ratio * target_frequency
    
    # Ensure voltage and frequency do not exceed rated values (important for VFDs)
    v_set = min(v_set, rated_voltage)
    target_frequency = min(target_frequency, rated_frequency)
    
    return v_set, target_frequency

In [13]:
# Instantiate the VFD and Motor Parameters (based on your problem) ---
vfd_config = VFDParams(
    rated_voltage_V=460.0,
    rated_frequency_Hz=60.0,
    rated_rpm=2000.0
    # Other control parameters will use their defaults
)

# Define the Target Operating Speed ---
TARGET_RPM = 1500.0

# Call the Rewritten Function ---
voltage_setting, frequency_setting = calculate_vfd_voltage_dataclass(
    target_rpm=TARGET_RPM,
    params=vfd_config
)

# Print Results ---
print(f"VFD Setting Calculation (with Dataclass)")
print(f"--------------------------------------------")
print(f"Motor Base Specs: {vfd_config.rated_voltage_V}V / {vfd_config.rated_frequency_Hz}Hz at {vfd_config.rated_rpm} RPM")
print(f"Target Speed: {TARGET_RPM:.1f} RPM")

print(f"Required VFD Output Voltage: {voltage_setting:.2f} V")
print(f"Required VFD Output Frequency: {frequency_setting:.2f} Hz")

VFD Setting Calculation (with Dataclass)
--------------------------------------------
Motor Base Specs: 460.0V / 60.0Hz at 2000.0 RPM
Target Speed: 1500.0 RPM
Required VFD Output Voltage: 345.00 V
Required VFD Output Frequency: 45.00 Hz


#### Operating Point Computations and using @dataclass

In [7]:
@dataclass
class HydraulicParams:
    """Holds the constant coefficients for the pump and system curves."""
    N_DESIGN: float     # Pump design/rated rotational speed (RPM)
    H0: float           # Pump shut-off head at N_DESIGN (m)
    K_PUMP: float       # Pump head curve coefficient (H_pump = H0 - K_PUMP * Q^2)
    H_STATIC: float     # System static head (m)
    K_SYS: float        # System friction coefficient (H_loss = K_SYS * Q^2)

In [8]:
def calculate_pump_operating_point_dataclass(
    rpm: float,
    params: HydraulicParams
) -> tuple[float, float]:
    
    # Access parameters via the object 'params'
    N_DESIGN = params.N_DESIGN
    H0 = params.H0
    H_STATIC = params.H_STATIC
    K_PUMP = params.K_PUMP
    K_SYS = params.K_SYS

    # Apply Affinity Law (N^2 scaling) and combine head terms
    ratio2 = (rpm / N_DESIGN) ** 2
    
    # Left side of the rearranged equation: H0 * (N/N_DESIGN)^2 - H_STATIC
    left = H0 * ratio2 - H_STATIC
    
    # Denominator: K_PUMP + K_SYS
    denom = K_PUMP + K_SYS
    
    # Solve for Q
    if left <= 0.0 or denom <= 0.0:
        Q = 0.0
    else:
        # Q = sqrt( left / denom )
        Q = math.sqrt(left / denom)
    
    # Calculate the resulting system head (H)
    H = H_STATIC + K_SYS * Q * Q
    
    return Q, H

In [9]:
# Instantiate the Hydraulic Parameters ---
# Define the constants for the specific pump and piping system
pump_system_params = HydraulicParams(
    N_DESIGN=1750.0,   # Rated Speed (RPM)
    H0=40.0,           # Shut-off Head at Rated Speed (m)
    K_PUMP=0.005,      # Pump curve slope coefficient
    H_STATIC=15.0,     # System Static Head (m)
    K_SYS=0.002        # System Friction coefficient
)

# Define the Target Operating Speed ---
TARGET_RPM = 1500.0 # The speed at which the VFD is running (RPM)

# Call the Rewritten Function ---
flow_rate, operating_head = calculate_pump_operating_point_dataclass(
    rpm=TARGET_RPM,
    params=pump_system_params
)

# Print Results ---
print(f"**Pump Operating Point Calculation (with Dataclass)**")
print(f"---------------------------------------------------")
print(f"Target Speed (N): {TARGET_RPM:.1f} RPM")
print(f"Calculated Flow Rate (Q): {flow_rate:.2f} m³/h")
print(f"Calculated Operating Head (H): {operating_head:.2f} m")

**Pump Operating Point Calculation (with Dataclass)**
---------------------------------------------------
Target Speed (N): 1500.0 RPM
Calculated Flow Rate (Q): 45.34 m³/h
Calculated Operating Head (H): 19.11 m


#### Example of data ingestion 

In [14]:
def ingest_head_stream(heads, dt=DT):
    """Yield (t, H_meas) pairs at fixed dt."""
    t = 0.0
    for h in heads:
        yield t, float(h)
        t += dt

heads = [20,22,25]

h = ingest_head_stream(heads,DT)

for i in h:
    print(i)

(0.0, 20.0)
(0.1, 22.0)
(0.2, 25.0)


In [ ]:
# Example: heads measured every 0.1 s
heads = [22.5, 22.6, 22.8, ...]  # your plant data
head_stream = ingest_head_stream(heads, dt=0.1)

log, twin = run_simulation(
    setpoint_head=25.0,
    sim_time_s=len(heads)*0.1,
    external_head_stream=head_stream
)


In [ ]:
# Baseline run could look like this
log, twin = run_simulation(H_sp=HEAD_SP, sim_time_s=SIM_TIME_S, params=TwinParams())
df = pd.DataFrame(log)
plot_basic_timeseries(df, f"Baseline (H_sp={HEAD_SP} m)")